# 5.Основы парсинга и работы с API

In [2]:
import pandas as pd
from numpy import NaN
import requests
from bs4 import BeautifulSoup
import time

### Задание 1
Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

def parser(queries: list) -> pd.DataFrame:
    url = 'https://habr.com/ru/search/'
    blog = pd.DataFrame()
    for query in queries:
        sleep(3)
        req = requests.get(url=url, params={'q': query})
        soup = BeautifulSoup(req.text)
        news = soup.find_all('article', class_='tm-articles-list__item')
        for el in news:
            try:
                title = el.find('h2', class_='tm-title tm-title_h2').text
                date = el.find('span', class_= 'tm-article-datetime-published').find('time').get('datetime')
                link = 'https://habr.com' + (el.find('h2', class_='tm-title tm-title_h2')
                                             .find('a', class_='tm-title__link').get('href'))
            except AttributeError:
                title = el.find('h2', class_='tm-megapost-snippet__title').text
                date = (el.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__date')
                        .find('time', class_='tm-megapost-snippet__datetime-published')
                        .get('datetime'))
                link = 'https://habr.com' + (el.find('a', class_='tm-megapost-snippet__link tm-megapost-snippet__card')
                                             .get('href'))
            row = {'date': date, 'title': title, 'link': link}
            blog = pd.concat([blog, pd.DataFrame([row])])
    return blog.reset_index(drop=True)

parser(['python', 'анализ данных'])

,date,title,link
0,2021-12-13T06:00:03.000Z,Жаждущим автоматизации: открытый урок «ChatOps...,https://habr.com/ru/companies/southbridge/news...
1,2022-01-20T15:37:16.000Z,Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/companies/southbridge/news...
2,2020-04-21T15:35:14.000Z,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/498364/
3,2021-07-06T10:29:57.000Z,Python Community Meetup 8/07: видео и материал...,https://habr.com/ru/companies/raiffeisenbank/n...
4,2022-01-13T15:35:30.000Z,Открытый урок «Пишем Custom Prometheus Exporte...,https://habr.com/ru/companies/southbridge/news...
5,2020-12-04T18:03:25.000Z,Python как компилируемый статически типизирова...,https://habr.com/ru/news/531402/
6,2022-03-08T09:13:52.000Z,Вышел мартовский релиз расширения Python для V...,https://habr.com/ru/news/654707/
7,2020-03-03T10:22:32.000Z,В начале этого года Python сместил Java и стал...,https://habr.com/ru/companies/itsumma/news/490...
8,2021-11-16T13:09:46.000Z,EPAM разработала бесплатный курс по программир...,https://habr.com/ru/companies/epam_systems/new...
9,2022-12-26T11:24:49.000Z,Positive Technologies открыла доступ по запрос...,https://habr.com/ru/news/707698/


### Задание 2.

<b>Обязательная часть</b>

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Check. Список email-ов задаем переменной в начале кода:

<i>EMAIL = [xxx@x.ru, yyy@y.com]</i>

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

<b><small>Подсказка: сервис работает при помощи “скрытого” API. Внимательно изучите post-запросы.</small></b>

In [6]:
def leaks():
  url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
  payload = {"emailAddresses": ['xxx@x.ru', 'yyy@y.com']}
  headers = {
      'Vaar-Header-App-Build-Version': '1.0.0',
      'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
      'Vaar-Version': '0'
      }
  req = requests.post(url=url, json=payload, headers=headers)
  df = pd.DataFrame(req.json()['breaches']).T.reset_index()
  return df[['publishDate', 'site', 'description']]

In [7]:
leaks()

,publishDate,site,description
0,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
1,2021-02-11T00:00:00Z,forums.vkmonline.com,"At an unconfirmed date, the Russian-language m..."
2,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...
3,2019-10-17T00:00:00Z,zynga.com,"In September 2019, the game developer Zynga wa..."
4,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
5,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
6,2018-02-18T00:00:00Z,netlog.com,Netlog (formerly known as Facebox and Bingbox)...
7,2017-03-15T00:00:00Z,globalreach.eu,"In 2016, Global Reach Technology's database wa..."
8,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
9,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
